<a href="https://colab.research.google.com/github/liamhall64/Habitable-Exomoons/blob/main/Detectability_and_Probability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
from astropy import constants as const

!pip install pytransit celerite emcee corner
from pytransit import QuadraticModel
# from pytransit import UniformModel
from scipy.optimize import minimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.0/308.0 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.1/698.1 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 946.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.5 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/pytools/persistent_dict.py:63: RecommendedHashNotFoundWarning: Unable to import recommended hash 'siphash24.siphash13', falling back to 'hashlib.sha256'. Run 'python3 -m pip install siphash24' to install the recommended hash.
  warn("Unable to import recommended hash 'siphash24.siphash13', "


# TTV calculations

In [ ]:
def barycentre(Mp, Mm, dpm, moon_period):
  dp = dpm/(1+(Mp/Mm)) # DIST BARYCENTRE TO PLANET
  dm = dpm/(1+(Mm/Mp)) # DIST BARYCENTRE TO MOON
  print(f'Planet-barycentre distance = {dp:.3f}\nMoon-barycentre distance = {dm:.3f}')
  return(dp, dm)

# PARAMETERS
Mp = 1*const.M_earth
Mm = 0.0123*const.M_earth
dpm = 0.3844e9*u.m
moon_period = 27.322*u.day
P = 1*u.year
P = P.to(u.day)

dp, dm = barycentre(Mp, Mm, dpm, moon_period)

Planet-barycentre distance = 4670670.750 m
Moon-barycentre distance = 379729329.250 m


In [ ]:
N_trials = 10 # NUMBER OF ANGLES
N_obs = 10 # NUMBER OF OBSERVATIONS/TRANSITS (SHOULD BE WITHIN 5 YEARS AT LEAST)

In [ ]:
t = np.linspace(0, 2*P, 1000)

# BARYCENTRE
ab = 1*const.au # SEPARATION
Pb = P.to(u.s)  # PERIOD
wb = (2*np.pi/Pb) *u.rad # ANGULAR FREQ
alphab = wb*t
xb = ab*np.sin(alphab)

angle = []
def position(N_trials):
  for i in range(N_trials):
    angle.append(np.random.uniform(0, 2*np.pi)*u.rad)
    #print(f"Start angle #{i+1} = {angle[i]:.3f}") # IF WE WANT TO PRINT THE LIST OF ALL STARTING ANGLES OF 'N' REALISATIONS

    # PLANET
    wp = 2*np.pi/moon_period *u.rad
    alphap = wp*t + angle[i]
    xp = dp * np.sin(alphap) + xb

    # MOON
    wm = 2*np.pi/moon_period *u.rad
    alpham = wm*t + np.pi*u.rad + angle[i]
    xm = dm*np.sin(alpham) + xb
  return angle, xb, xp, xm, wb, wp, wm, ab, alphap, alpham


angle, xb, xp, xm, wb, wp, wm, ab, alphap, alpham = position(N_trials)
print('Done')

Done


In [ ]:
t = np.arange(0,N_obs*P.value,P.value)*P.unit # TIME

# BARYCENTRE
ab = 1*const.au # SEPARATION
Pb = P.to(u.s)  # PERIOD
wb = (2*np.pi/Pb) *u.rad # ANGULAR FREQ
alphab = wb*t
xb = ab*np.sin(alphab)

ttvp = []
det = 0

for i in range(N_trials):
  angle = np.random.uniform(0, 2*np.pi)*u.rad

  # PLANET
  wp = 2*np.pi/moon_period *u.rad
  alphap = wp*t + angle
  xp = dp * np.sin(alphap) + xb
  diffp = xb - xp

  vb = (2*np.pi*ab)/P.to(u.s) # BARYCENTRE VELOCITY
  #print(f'Velocity of system around star = {vb:.3f}')
  ttvp.append(diffp/vb)
  if N_trials <=10:
    print(ttvp[i])
else:
  print('Too many to print!')
ttvp = ttvp[i].to(u.day)


[  95.62783773 -156.20803677  115.86007199   -0.6533191  -114.97555226
  156.31713588  -96.66006525  -25.45040472  131.11701771 -152.06711983] s
[-122.62587232  154.94947048  -87.15808183  -36.94734382  137.18058179
 -148.77955664   64.250011     61.7923516  -147.90982483  138.46072326] s
[  32.99742725   90.49714804 -155.52026967  120.05960328   -7.02677377
 -110.54615104  156.69368525 -101.59927146  -19.13983082  127.51242481] s
[  44.98616102 -141.02041564  145.93943761  -56.5649791   -69.35689141
  150.46631921 -134.35738598   31.43829532   91.7935109  -155.71626557] s
[  11.28864667 -122.76112586  154.91595099  -86.97744671  -37.15838372
  137.28567069 -148.71079516   64.05182687   61.99190887 -147.98181855] s
[ 147.8005812   -61.49080813  -64.54902292  148.8828413  -137.02140554
   36.62855263   87.43051258 -154.99951953  122.42120229  -10.7448603 ] s
[  92.89494158   30.10198974 -133.64961094  150.84437791  -70.57651492
  -55.29180632  145.43533004 -141.61108411   46.2899665    

# Light curve

In [ ]:
# SWAPPED THE t0 FOR ttvp FOR THE TIME OF INFERIOR CONJUNCTION IN DAYS TO RUN IN A LOOP TO GET MULTIPLE LIGHT CURVES
per = 365.25                         #orbital period in days
rp_rs = const.R_earth/const.R_sun        #planet radius / stellar radius ratio
ars =  const.au/const.R_sun              #semi-major axis / stellar radius ratio
inc =  (90*u.deg).to(u.rad).value    #orbital inclination (in radians)
ecc = 0.                             #eccentricity
w = (90*u.deg).to(u.rad).value       #longitude of periastron (in radians)
gamma = [0.231, 0.226]                 #limb darkening coefficients [u1, u2]

texp = 100*u.s.to(u.day)             # the exposure time : we convert seconds to days
t = np.arange(-0.5, 0.5, texp)       # new timegrid #note we use np.arange rather than np.linspace (DAYS) MANDEL AND........
tm = QuadraticModel()                # a model that uses two limb-darkening coefficients
tm.set_data(t)

lc_Earth = []
for j in range(len(ttvp)):
  # ttvp[j] is a scalar Quantity, so we directly use it as t0_val
  t0_val = ttvp[j]
  lc_Earth.append(tm.evaluate(k=rp_rs, ldc=gamma, t0=t0_val, p=per, a=ars, i=inc, e=ecc, w=w))
  #plt.plot(t, lc_Earth[j], '-o') # Plot the last added light curve
print('Done')

Done


In [ ]:
def planck(wl, T):
  a = np.float64(1.191042768e8)*u.um**5 *u.W/ u.m**2 /u.sr/u.um
  b = np.float64(14387.7516)*1*u.um * 1*u.K
  try:
    x = b/(wl*T)
    bb = a/wl**5 / (np.exp(x) - 1.0)
  except ArithmeticError:
    bb = np.zeros(np.size(wl))
  return bb

s_lum, s_rad = 1*const.L_sun, 1*const.R_sun              # STELLAR VALUES
T_s = (s_lum/(4*np.pi*s_rad**2*const.sigma_sb))**0.25    # TEMP-LUMINOSITY RELATIONSHIP

wl = np.linspace(0.6, 5.3, 1000)*u.um                    # JWST NIRSPEC PRISM WAVELENGTH RANGE (OR HUBBLE 1.1-1.7)
wl_full = np.linspace(0,6, 1000)*u.um
BB_flux = np.pi*u.sr*planck(wl,T_s)                      # FLUX DENSITY - W/M^2/MICRONS
BB_flux_full = np.pi*u.sr*planck(wl_full,T_s)

####

R_s = 1*u.Rsun.to(u.m) # host star radius in Rsun (code converts this to m)
d = 10*u.pc.to(u.m) # distance to star in pc (code converts this to m)

Flux = BB_flux*(R_s/d)**2

D = 6.5*u.m; Atel = np.pi*(D/2)**2 # APERTURE OF SPECIFIC TELESCOPE
trans = 0.5; QE = 0.8 # QUANTUM EFFICIENCY
Power_per_micron = Flux*Atel*trans*QE # WATTS / MICRON
Power = Power_per_micron*np.gradient(wl) # WATTS

Photons_per_second = Power/(const.h*const.c/wl.to(u.m)) # CONVERT WATTS TO PHOTONS
Photons_per_second = np.sum(Photons_per_second).value*1/u.s # TOTAL ELECTRONS PER SECOND

exposure_time = 100*u.s # GENRALISED EXPOSURE
Electrons = (Photons_per_second*exposure_time).value
std = Electrons**0.5

Noise = Electrons**0.5 # ASSUMING A NOISE LIMITED INSTRUMENT
Noise = np.random.normal(0, Noise, len(lc_Earth[0]))

'''
for i in range(len(lc_Earth)):
  lc = lc_Earth[0]*Electrons
  lc = lc + Noise
  plt.plot(t, lc, '-o', label = f'transit {i+1}')
  plt.grid()
  plt.title('Noise light curves for each ')
  plt.ylabel('Relative signal')
  plt.xlabel('Time (days)')

  if N_obs <=10:
    plt.legend()
'''

"\nfor i in range(len(lc_Earth)):\n  lc = lc_Earth[0]*Electrons\n  lc = lc + Noise\n  plt.plot(t, lc, '-o', label = f'transit {i+1}')\n  plt.grid()\n  plt.title('Noise light curves for each ')\n  plt.ylabel('Relative signal')\n  plt.xlabel('Time (days)')\n\n  if N_obs <=10:\n    plt.legend()\n"

In [ ]:
def chi_squared(X, lc, Noise):
    rp_rs = X[0]
    S = X[1]
    t_0 = X[2]
    model =tm.evaluate(k=rp_rs, ldc=gamma, t0=t_0, p=per, a=ars, i=inc, e=ecc, w=w) *S
    return np.sum(((model-lc))**2/Noise**2)

t0 = ttvp
data_lc = lc
S= Electrons

fit_init = [((lc.max()-lc.min())/ lc.max())**0.5, np.mean(lc[0:40]), t0[0].value+np.random.normal(0,0.001)] # these are initial guesses - we'll cheat a bit by putting in the known values but you can try starting with different initial values too
# Access the first element of t0.value to ensure it's a scalar
bounds =((rp_rs*0.99,rp_rs*1.1), (S*0.99,S*1.1), (np.min(ttvp.value),np.max(ttvp.value))) # the bounds over which the algorithm will vary the parameters
# Use std instead of Noise
fit  = minimize(chi_squared, fit_init, args=(data_lc, std), method='Nelder-Mead', jac=None, hess=None, hessp=None, bounds=bounds, constraints=(), tol=None, callback=None, options=None)
final_fit = [fit['x'][0], fit['x'][1], fit['x'][2]]
if N_obs <= 10:
  print ('Fitted Rp/Rs', final_fit[0], 'Actual Rp/Rs', rp_rs)
  print ('Fitted S', final_fit[1], 'Actual S', S)
  print ('Fitted t0', final_fit[2], 'Actual t0', t0)

model_fit = tm.evaluate(k=fit['x'][0], ldc=gamma, t0=fit['x'][2], p=per, a=ars, i=inc, e=ecc, w=w) *fit['x'][1]
'''
plt.figure('lc')
plt.plot(t,data_lc, 'o')
plt.plot(t,model_fit, '-')
plt.show();
'''
print('Done')

NameError: name 'lc' is not defined

In [ ]:
t0_list = []

for i in range(N_obs):
  Noise = Electrons**0.5 # ASSUMING A NOISE LIMITED INSTRUMENT
  Noise = (np.random.normal(0, Noise, len(lc_Earth[0])))

  lc = lc_Earth[0]*Electrons
  lc = lc + Noise

  if N_obs <=10:
    plt.plot(t, lc, '-o')
    plt.grid()
    plt.title('Noise light curves for each ttv')
    plt.ylabel('Relative signal')
    plt.xlabel('Time (days)')

  t0 = ttvp
  data_lc = lc
  S = Electrons

  fit_init = [((lc.max()-lc.min())/ lc.max())**0.5, np.mean(lc[0:40]), t0[0].value+np.random.normal(0,0.001)] # these are initial guesses - we'll cheat a bit by putting in the known values but you can try starting with different initial values too
  # Access the first element of t0.value to ensure it's a scalar
  bounds =((rp_rs*0.99,rp_rs*1.1), (S*0.99,S*1.1), (-0.01,0.01)) # the bounds over which the algorithm will vary the parameters
  # Use std instead of Noise
  fit  = minimize(chi_squared, fit_init, args=(data_lc, std), method='Nelder-Mead', jac=None, hess=None, hessp=None, bounds=bounds, constraints=(), tol=None, callback=None, options=None)
  final_fit = [fit['x'][0], fit['x'][1], fit['x'][2]]
  #print ('Fitted Rp/Rs', final_fit[0], 'Actual Rp/Rs', rp_rs_Earth)
  #print ('Fitted S', final_fit[1], 'Actual S', S)
  #print ('Fitted t0', final_fit[2], 'Actual t0', t0)

  model_fit = tm.evaluate(k=fit['x'][0], ldc=gamma, t0=fit['x'][2], p=per, a=ars, i=inc, e=ecc, w=w) * fit['x'][1]

  #plt.figure('lc')
  #plt.plot(t,data_lc, 'o')
  #plt.plot(t,model_fit, '-')

  t0_list.append(final_fit[2])

#plt.show()

# Probability

This calculates the SNR of all light curves and we should expect that for Earth, each transit or observation there should be a detection of a transit timing variation due to size of the Moon and separation.

In [ ]:
t0_list_sec = [t*u.day.to(u.s) for t in t0_list]
stddev = np.std(t0_list_sec)*u.s

det = 0

for i in range(len(ttvp)):
  SNR = np.abs(ttvp[i]/stddev)
  #print(SNR) # IF ANY ARE GREATER THAN 5*SIGMA THEN SUCCESSFUL DETECTION

  if SNR >= 5:
    if N_obs <=10:
      print('Detection')
    det = det+1
  else:
    print(f'Obs {i+1} = No detection')

prob = 100*det/N_obs
print(f'Probability of detection out of {N_obs} transits = {prob:.1f}%')

Obs 10 = No detection
Obs 29 = No detection
Obs 48 = No detection
Obs 67 = No detection
Obs 86 = No detection
Obs 215 = No detection
Obs 234 = No detection
Obs 253 = No detection
Obs 272 = No detection
Obs 291 = No detection
Obs 310 = No detection
Obs 329 = No detection
Obs 348 = No detection
Obs 367 = No detection
Obs 386 = No detection
Obs 405 = No detection
Obs 424 = No detection
Obs 443 = No detection
Obs 462 = No detection
Obs 572 = No detection
Obs 591 = No detection
Obs 610 = No detection
Obs 629 = No detection
Obs 648 = No detection
Obs 667 = No detection
Obs 686 = No detection
Obs 705 = No detection
Obs 724 = No detection
Obs 743 = No detection
Obs 762 = No detection
Obs 781 = No detection
Obs 800 = No detection
Obs 819 = No detection
Obs 929 = No detection
Obs 948 = No detection
Obs 967 = No detection
Obs 986 = No detection
Probability of detection out of 1000 transits = 96%
